testing keras custom layer stuff

In [356]:
# import packages:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import os 
import tensorflow as tf
from tensorflow.keras import Input, Model, regularizers, constraints, layers, optimizers
from keras.layers import Layer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

define classes:

In [357]:
# randomizer seed:
np.random.seed(0)

# need to define a constraint for training the parameters:
class OrderedConstraint(constraints.Constraint):
    # constructor:
    def __init__(self):
        pass

    # call function for constraint:
    def __call__(self, W):
        return tf.sort(W, axis = 2)

# first layer: 
class MF_Layer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, **kwargs):
        super(MF_Layer, self).__init__(**kwargs)
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs

        # need to initialize antecedent parameters, but a <= b <= c. therefore 
        # generate a set of "raw parameters" to be sorted. these are not trained

        raw_params = np.random.uniform(low = 0.0, high = 50.0, size = (self.num_inputs, self.num_mfs, 3))
        sorted_params = tf.sort(raw_params, axis = -1)

        self.mf_params = self.add_weight(
            shape = (self.num_inputs, self.num_mfs, 3),             # num_inputs, num_mfs per input, 3 params per mf (triangular)
            initializer = tf.constant_initializer(sorted_params.numpy()),
            trainable = True,
            constraint = OrderedConstraint(),
            name = 'Antecedent_Params'
        )

    # # custom setting of weights:
    # def set_weights(self, input_index, mf_index, weights):
    #     # this function is used to set weights based on what a user provides
    #     # user must provide weights in the form of a np.array of shape (3,)
        

    # function call:
    def call(self, inputs):
        # need to initialize the membership values:
        membership_values = []

        # for every input:
        for i in range(self.num_inputs):
            # get the memberships for that input:
            input_mf_params = self.mf_params[i]

            # need to now compute the fuzzified value for each membership function:
            fuzzified_values = []

            # for every membership function:
            for j in range(self.num_mfs):
                a = input_mf_params[j, 0]
                b = input_mf_params[j, 1]
                c = input_mf_params[j, 2]

                left = (inputs[:, i] - a) / (b - a)     
                right = (c - inputs[:, i]) / (c - b)

                output = tf.maximum(0.0, tf.minimum(left, right))

                fuzzified_values.append(output)
        
            # need to now stack the mf values for that given input:
            membership_values.append(tf.stack(fuzzified_values, axis = -1))

        # stack everything and return:
        return tf.stack(membership_values, axis = 1)


values for making model:

model testing:

In [358]:
model = MF_Layer(3,3)

weights = np.array([3, 1, 4])



[3 1 4]
